#06_SparkDataAnal.ipynb
TLC Trip Record Data
출처: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [2]:
df  = spark.read.format('json')\
    .load('learning_spark_data/2015-summary.json')

In [3]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [4]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [8]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [10]:
df.select('count').show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [13]:
# 도착국가명 중복제거
df.select('DEST_COUNTRY_NAME').distinct().show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



In [15]:
df1 = df.select('DEST_COUNTRY_NAME').distinct().cache()
df1.count()

132

In [16]:
# row class 를 이용한 단일 레코드 생성
from pyspark.sql import Row
myRow = Row("hello", None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [19]:
# 새로운 컬럼 추가하기
from pyspark.sql.functions import expr

df3 = df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME')) # epxr sql표현식을 받아 생성
df3

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, withinCountry: boolean]

In [20]:
df3.show(3)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
+-----------------+-------------------+-----+-------------+
only showing top 3 rows



In [26]:
df3.filter(df3['withinCountry'] == True).show(5)

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [30]:
df3.createOrReplaceTempView("df3_view")

query = '''
SELECT *
FROM df3_view
WHERE withinCountry = True
'''
spark.sql(query).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



# 강사님 버전
df4 = df.withColumn('Category', expr("CASE WHEN count<10 THEN 'under' WHEN count>=10 THEN 'upper' END"))
df4.show()

In [31]:
#case when 카운트 10 이하 under, 이상 upper로 변환 > category 컬럼 추가
from pyspark.sql.functions import when

df3 = df3.withColumn(
    "category",
    when(df3["count"] <= 10, "under").otherwise("upper")
)

In [32]:
df3.show(3)

+-----------------+-------------------+-----+-------------+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|category|
+-----------------+-------------------+-----+-------------+--------+
|    United States|            Romania|   15|        false|   upper|
|    United States|            Croatia|    1|        false|   under|
|    United States|            Ireland|  344|        false|   upper|
+-----------------+-------------------+-----+-------------+--------+
only showing top 3 rows



In [33]:
query = '''
SELECT *,
       CASE 
           WHEN count <= 10 THEN 'under'
           ELSE 'upper'
       END AS category
FROM df3_view
'''
spark.sql(query).show(3)

+-----------------+-------------------+-----+-------------+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|category|
+-----------------+-------------------+-----+-------------+--------+
|    United States|            Romania|   15|        false|   upper|
|    United States|            Croatia|    1|        false|   under|
|    United States|            Ireland|  344|        false|   upper|
+-----------------+-------------------+-----+-------------+--------+
only showing top 3 rows



In [37]:
spark.stop()

In [39]:
spark = SparkSession.builder \
    .master("local")\
    .appName("emp") \
    .getOrCreate()

In [46]:
dept_df = spark.read.format('csv')\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .load('learning_spark_data/dept.csv')

In [57]:
emp_df = spark.read.format('csv')\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .load('learning_spark_data/emp.csv')

In [58]:
emp_df.select('*').show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
| 9292|  JACK|

In [59]:
emp_df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [60]:
from pyspark.sql.functions import countDistinct
emp_df.select( countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [61]:
from pyspark.sql.functions import approx_count_distinct
emp_df.select( approx_count_distinct('job', 0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [64]:
# first, last, min, max, sum, avg -> (expr: sql문장 x) function으로 처리

In [69]:
from pyspark.sql.functions import first, last, min, max, sum, avg, count

In [73]:
emp_df.select(count('sal')).show()

+----------+
|count(sal)|
+----------+
|        15|
+----------+



In [72]:
emp_df.select(first('sal')).show()

+----------+
|first(sal)|
+----------+
|       800|
+----------+



In [74]:
emp_df.select(max('sal')).show()

+--------+
|max(sal)|
+--------+
|    5000|
+--------+



In [75]:
emp_df.select(sum('sal')).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [88]:
emp_df.select(expr('count (sal) as total_transaction')).show()

+-----------------+
|total_transaction|
+-----------------+
|               15|
+-----------------+



In [86]:
#total_salary / total_transaction, avg_salary, mean_salary
emp_df.selectExpr(
    "count(sal) as total_transaction",
    "round(avg(sal),2) as avg_salary",
    "round(mean(sal),2) as mean_salary"
).show()

+-----------------+----------+-----------+
|total_transaction|avg_salary|mean_salary|
+-----------------+----------+-----------+
|               15|   2148.33|    2148.33|
+-----------------+----------+-----------+



In [87]:
# 그룹화
emp_df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [92]:
emp_df.groupBy('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
         sum('sal')
).show()

+---------+---+----------+--------+
|      job|qty|count(job)|sum(sal)|
+---------+---+----------+--------+
|  ANALYST|  2|         2|    6000|
| SALESMAN|  4|         4|    5600|
|    CLERK|  5|         5|    7350|
|  MANAGER|  3|         3|    8275|
|PRESIDENT|  1|         1|    5000|
+---------+---+----------+--------+



In [97]:
# sal의 평균(SAL_AVG), 표준편차(SAL_STDEV)를 job별로 계산해서 출력,소수점 두자리
emp_df.groupBy('job').agg(
    expr('round(avg(sal),2) as SAL_AVG') ,
    expr('round(stddev(sal),2) as SAL_STDEV')
).show()

+---------+-------+---------+
|      job|SAL_AVG|SAL_STDEV|
+---------+-------+---------+
|  ANALYST| 3000.0|      0.0|
| SALESMAN| 1400.0|   177.95|
|    CLERK| 1470.0|   984.63|
|  MANAGER|2758.33|   274.24|
|PRESIDENT| 5000.0|     NULL|
+---------+-------+---------+



In [108]:
from pyspark.sql.functions import rank, desc

In [114]:
# 급여 top 10 보내기
from pyspark.sql.window import Window
windowspec = Window.orderBy(desc('sal'))
salAllRank = rank().over(windowspec)
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [115]:
emp_df.show(5)

+-----+------+--------+----+----------+----+----+------+
|empno| ename|     job| mgr|  hiredate| sal|comm|deptno|
+-----+------+--------+----+----------+----+----+------+
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES| MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN|SALESMAN|7698|1981-09-28|1250|1400|    30|
+-----+------+--------+----+----------+----+----+------+
only showing top 5 rows



In [116]:
emp_df.withColumn('salary_rank', salAllRank).show(5)

+-----+-----+---------+----+----------+----+----+------+-----------+
|empno|ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+-----+---------+----+----------+----+----+------+-----------+
| 7839| KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292| JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788|SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
| 7902| FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          3|
| 7566|JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          5|
+-----+-----+---------+----+----------+----+----+------+-----------+
only showing top 5 rows



In [130]:
# 1. 직무별로 급여 내림차순 정렬 기준의 윈도우 정의
windowSpec = Window.partitionBy("job").orderBy(desc("sal"))

# 2. rank() 함수 적용하여 직무별 순위 매기기
job_rank_df = emp_df.withColumn("rank", rank().over(windowSpec))

# 3. 결과 확인
job_rank_df.show(3)

+-----+-----+-------+----+----------+----+----+------+----+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|rank|
+-----+-----+-------+----+----------+----+----+------+----+
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|   1|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|   1|
| 9292| JACK|  CLERK|7782|1982-01-23|3200|NULL|    70|   1|
+-----+-----+-------+----+----------+----+----+------+----+
only showing top 3 rows



In [127]:
ndf = emp_df.groupBy('job').agg(expr('sum(sal) as sumsal'))
windowspec = Window.orderBy(desc('sumsal'))
salAllRank = rank().over(windowspec)
salAllRank
ndf.withColumn('salary_rank', salAllRank).show(5)

+---------+------+-----------+
|      job|sumsal|salary_rank|
+---------+------+-----------+
|  MANAGER|  8275|          1|
|    CLERK|  7350|          2|
|  ANALYST|  6000|          3|
| SALESMAN|  5600|          4|
|PRESIDENT|  5000|          5|
+---------+------+-----------+

